# Ranking evaluation metrics

In this lab you will implement ranking evaluation metrics.

For each query the search engine returns the sorted list of documents. We expect to have relevant documents at the top. 
In supervision learning to evaluate the quality we need labeled data.

Read this article first:
http://queirozf.com/entries/evaluation-metrics-for-ranking-problems-introduction-and-examples
https://medium.com/swlh/rank-aware-recsys-evaluation-metrics-5191bba16832

## Binary relevance metrics

We will start from the assumption, that each document is either relevant or not.

#### recall, precision, f1

First metrics are already faminiar for you from Introduction to Machine Learning course.
Implement relall, precision, and relevance for top $k$ documents.

relevance is a list which represents that document with this index in ranking is relevant or not.

In [0]:
import numpy as np
import math

def recall_at_k(relevance, k):
    # Your code here

    doc = relevance[:k]
    doc_rest = relevance[k:]
    tp = sum([t==1 for t in doc])
    fn = sum([t==1 for t in doc_rest])

    return tp / (tp + fn)

def precision_at_k(relevance, k):
    # Your code here

    doc = relevance[:k]
    doc_rest = relevance[k:]
    tp = [t==1 for t in doc]
    fp = len(tp) - sum(tp)
    tp = sum(tp)

    return tp / (tp + fp)

def f1_at_k(relevance, k):
    # Your code here

    prec = precision_at_k(relevance, k)
    recal = recall_at_k(relevance, k)

    return 2 * (prec * recal) / (prec + recal)

In [0]:
r = [1, 0, 1, 1, 0, 1, 0, 0]

print(recall_at_k(r, 1))
print(recall_at_k(r, 8))

print(precision_at_k(r, 1))
print(precision_at_k(r, 8))

print(f1_at_k(r, 1))
print(f1_at_k(r, 8))

0.25
1.0
1.0
0.5
0.4
0.6666666666666666


#### Average Precision
You can calculate the AP using the following algorithm:

<img src="http://queirozf.com/images/contents/mnc7sx1.png">

In [0]:
def average_precision(relevance, k):
    # Your code here

    correct_pred = 0
    running_sum = 0

    for i in range(0, k):
      if relevance[i] == 1:
        correct_pred += 1
        running_sum += (correct_pred / (i+1))

    return running_sum / correct_pred

In [0]:
print(average_precision(r, 1))
print(average_precision(r, 8))

1.0
0.7708333333333333


## Relevance as a real number

#### DCG

DCG - discounted cumulative gain, does't require the relevance to be a binary feature. In many situations one document is more relevant than another and we want to represent it in supervised evaluation. Often the relevance is a number form ${0,1,2,3}$, but ${0,1}$ is also appropriate for usage.

The idea is that each rerelvant document brings a "gain" for a user. He or she looks through the documents from the first. So the gain sums cumulatively. But it is better to have a relevant document at the top of the ranking, so the weight of that gain decreases, or we have a discounded weight with increasing of the document position. And since the weight is decreasing, we can calculate this value for the top $k$ documents in ranking.

$$DCG@k = \sum_{1}^{k}\frac{2^{rel_i}-1}{log_2(i+1)}$$

In [0]:
def dcg_at_k(relevance, k=10):
    # Your code here

    res = 0

    for i in range(0, k):
      res += (2**relevance[i] - 1) / math.log2(i + 2)

    return res

In [50]:
r2 = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]

print(dcg_at_k(r, 1))
print(dcg_at_k(r, 8))

print(dcg_at_k(r2, 1))
print(dcg_at_k(r2, 10))

1.0
2.2868837451814152
7.0
16.80260104782745


#### nDCG
Now the idea is to normalize it to the maximum value.

In [0]:
def ndcg_at_k(relevance, k=10):

    # Your code here

    dcg = dcg_at_k(relevance, k)
    best = list(filter(lambda x : x > 0, relevance[:k]))
    idcg = dcg_at_k(best, len(best))

    return dcg / idcg

In [56]:
print(ndcg_at_k(r, 1))
print(ndcg_at_k(r, 8))

print(ndcg_at_k(r2, 1))
print(ndcg_at_k(r2, 10))

1.0
0.8927537907700456
1.0
0.9664454855958076


## Test in real data

You already have search engines on songs or news. Test the evaluation on real data.

1. Choose the search query query
2. Run the search
3. Manually look top 10 results and evaluate each of them if it relevant or not
4. Calculate AP and DCG (relevance is either 0 or 1)

In [63]:
# Your code here

!wget https://archive.ics.uci.edu/ml/machine-learning-databases/reuters21578-mld/reuters21578.tar.gz > /dev/null 2> /dev/null
!tar xf reuters21578.tar.gz > /dev/null 2> /dev/null

import re, string, unicodedata
from sklearn.decomposition import PCA
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

def normalize(text):
    res = text.lower()
    res = re.sub(r'\d+', '', res)
    res = unicodedata.normalize('NFKD', res).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    res = re.sub(r'[^\w\s]', '', res)
    res = res.strip()
    return res
     
def tokenize(text):
    return nltk.word_tokenize(text)

def lemmatization(tokens):
    lemmer = WordNetLemmatizer()
    return [lemmer.lemmatize(word) for word in tokens]

def remove_stop_word(tokens):
    stop_words = set(stopwords.words('english'))
    return [i for i in tokens if not i in stop_words]

def preprocess(text):
    tokens = tokenize(normalize(text))
    tokens = lemmatization(tokens)
    tokens = remove_stop_word(tokens)
    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
import glob
from collections import Counter

def get_collection(docs=5, words=5):

    coll = []
    _docs = 0

    for _file in glob.glob("*.sgm"):

       with open(_file, 'r') as f:
          try:
            collection = f.read()
          except Exception as e:
            print(e)
            continue

       for i in range(len(collection.split("</REUTERS>"))):

         data =  collection.split("</REUTERS>")[i]
         data = data.split("<BODY>")[-1]
         data = data.split("</BODY>")[0]
         if _docs < docs:
           _docs += 1
           data = preprocess(data)
           coll +=  [data[:min(words, len(data))]]

    return coll

In [0]:
def make_index(collection):

    index = dict()
    word_freq = {}

    for i, doc in enumerate(collection):
      
        for w in doc:

            n = Counter(doc)[w]

            try:
              index[w][0] += n
              index[w].append((i, n))
            except KeyError:
              index[w] = [n, (i, n)]
            
    return index

In [60]:
collection = get_collection(100, 200)
index = make_index(collection)
doc_lengths = {k:len(v) for k, v in zip(range(len(collection)), collection)}

print('\n\n<> Index:')
print(index)
print('\n<> Doc lenghts:')
print(doc_lengths)

'utf-8' codec can't decode byte 0xfc in position 1519554: invalid start byte


<> Index:
{'president': [45, (0, 1), (4, 2), (4, 2), (25, 2), (25, 2), (33, 1), (48, 2), (48, 2), (58, 2), (58, 2), (71, 4), (71, 4), (71, 4), (71, 4), (84, 1), (86, 2), (86, 2), (96, 1), (97, 1), (98, 2), (98, 2)], 'jose': [1, (0, 1)], 'sarney': [4, (0, 2), (0, 2)], 'today': [9, (0, 1), (13, 1), (21, 1), (29, 1), (40, 1), (49, 1), (50, 1), (57, 1), (85, 1)], 'declared': [1, (0, 1)], 'war': [1, (0, 1)], 'without': [3, (0, 1), (42, 1), (48, 1)], 'quarter': [17, (0, 1), (15, 3), (15, 3), (15, 3), (31, 1), (60, 1), (83, 2), (83, 2), (85, 1)], 'inflation': [20, (0, 3), (0, 3), (0, 3), (4, 3), (4, 3), (4, 3), (8, 1), (84, 1)], 'said': [961, (0, 1), (2, 3), (2, 3), (2, 3), (3, 1), (4, 6), (4, 6), (4, 6), (4, 6), (4, 6), (4, 6), (5, 3), (5, 3), (5, 3), (6, 1), (7, 2), (7, 2), (8, 8), (8, 8), (8, 8), (8, 8), (8, 8), (8, 8), (8, 8), (8, 8), (11, 1), (13, 1), (14, 3), (14, 3), (14, 3), (15, 3), (15, 3), (15, 3), (16, 

In [0]:
import pandas as pd

words = {list(index.keys())[i] : i for i in range(len(index.keys()))}
td_matrix = np.zeros((len(doc_lengths.keys()), len(words)))

docs_num = len(doc_lengths.keys())
num_words = len(index.keys())

for w in index: 
    tf = sum([each[1] for each in index[w][1:]]) / num_words
    for doc_count in index[w][1:]:
        idf = math.log(docs_num / (1 + index[w][0]))
        td_matrix[doc_count[0]][words[w]] = tf * idf;

In [66]:
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity

def find_k_closest(query, dataset, k=5):    
    #TODO write here the code that will find 5 closest rows in dataset in terms of cosine similarity
    #HINT: as vectors in dataset are already normed, cosine similarity is just dot product.    
    
    dot_product = cosine_similarity(query.reshape(1, -1), dataset)
    mapped_list = [[index, dataset[index], dot_product[0][index]] for index in range(dot_product.size)]
    sorted_list = sorted(mapped_list, key=lambda vec: vec[2], reverse=True)
    return sorted_list[:k]

new_k = 5
new_pca = PCA(n_components=new_k)
new_min = new_pca.fit_transform(td_matrix)
new_norm = normalize(new_min)

print(new_pca.explained_variance_ratio_)
print(new_norm.shape)

[0.77409428 0.18701669 0.01945817 0.00311772 0.0020364 ]
(100, 5)


In [67]:
recommend_to = 0
r = find_k_closest(new_norm[recommend_to,:], new_norm)

print("For:", recommend_to)
for k, v, p in r:
    if recommend_to != k: # exclude itself 
        print("\t", k, "sim=", p)

For: 0
	 94 sim= 0.9941221672829404
	 85 sim= 0.9937965700325844
	 33 sim= 0.9933421734783769
	 50 sim= 0.993120482707687


In [79]:
def query_processing(query, index, new_norm):

  terms = index[query]
  terms = terms[1:]

  for indx, _ in set(terms):
    r = find_k_closest(new_norm[indx,:], new_norm, 10)

  return [(k, p) for k, _, p in r]

query = 'week'
responce = query_processing(query, index, new_norm) 
print(responce)

[(23, 1.0000000000000002), (32, 0.9999852646200745), (24, 0.9999575386013029), (14, 0.9955781518900981), (59, 0.9954527607208965), (50, 0.9946721241999896), (33, 0.9944812458267733), (85, 0.9938903847143133), (94, 0.9935774517555641), (0, 0.9802237749913238)]


In [77]:
r = [1, 0, 0, 1, 1, 1, 0, 0, 0, 1]
print('AP:', average_precision(r, 10))
print('DCG:', dcg_at_k(r, 10))

AP: 0.6533333333333333
DCG: 2.462801378733845
